# Types and Their Usefulness

We must all be familiar with types. They are fundamental to most of the programming languages that we have
seen thus far. Let us take a language such as Java.  Whenever we write a class, function definition or declare
a variable, we need to tell Java its type, even if obvious to us.

~~~
class Hello {
  public int y;
  public Hello (Hello h){
     ... constructor code ...
  }
  
  public String foo( int x) {
     .. code must return a String .. 
  
  }
~~~

You see that every declaration introduces the type of the thing that has been declared. `y` is declared to be an `int`. `h` the argument in the constructor of class `Hello` has type `Hello`. The return type of function `foo` is a `String` and its argument `x` has type `int`. 


What about other languages such as Python?

~~~
>>> x = 10
>>> print(type(x))
<class 'int'>
>>> y = {"x":10, "y": 35, "z":50}
>>> print(type(y))
<class 'dict'>
~~~

Even though we do not write types, it is clear that the Python interpreter has the notion of types. Furthermore, it checks during runtime if operations make sense depending on the type of the argument.

~~~
>>> x = 10
>>> y = "hello"
>>> z = y * x
>>> w = x ^ y
TypeError: unsupported operand type(s) for ^: 'int' and 'str'
~~~

## Benefits of static typing

Types can be checked at compile time (e.g. Java) or at run time (e.g. Python). If the former, the language is said to be *statically typed*; if the latter, the language is said to be *dynamically typed*.

A major benefit of static typing is that you can rule out many errors *before* running your program. Consider the following Python program, which asks for the user's name and greets them in reponse. Can you find the bug?

```python
def get_first_name(full_name):
    return full_name.split(" ")[0]

fallback_name = {
    "first_name": "UserFirstName",
    "last_name": "UserLastName"
}

raw_name = input("Please enter your name: ")
first_name = get_first_name(raw_name)

# If the user didn't type anything in, use the fallback name
if not first_name:
    first_name = get_first_name(fallback_name)

print(f"Hi, {first_name}!")
```

The bug is in the line: 

```
   first_name = get_first_name(fallback_name)

```

`fallback_name` is a dictionary whereas the function `get_first_name` expects its input to be a string. This is not going to show up under normal circumstances.

~~~
Please enter your name:Sriram Sankaranarayanan
Hi, Sriram!
~~~

However, if we typed in a different first name for instance the empty string `''`
We get an error:

~~~
Traceback (most recent call last):
 ....
AttributeError: 'dict' object has no attribute 'split'
~~~



This bug only manifests at run time and on a specific input (the empty string) -- finding and debugging such an error takes time and effort. A static type system would rule out such an error automatically without having to run your program since it would object passing a dictionary to a function that expects a string as an input. Recognizing this, Python 3.6 added support for type hints, which serve as a makeshift static type system for Python.

```python
from typing import Dict

def get_first_name(full_name: str) -> str:
    return full_name.split(" ")[0]

fallback_name: Dict[str, str] = {
    "first_name": "UserFirstName",
    "last_name": "UserLastName"
}

raw_name: str = input("Please enter your name: ")
first_name: str = get_first_name(raw_name)

# If the user didn't type anything in, use the fallback name
if not first_name:
    first_name = get_first_name(fallback_name)

print(f"Hi, {first_name}!")
```
This enables the type checker to catch the bug before running the code. In a language-aware IDE such as PyCharm, you can even catch type errors as you write the code, just as a spell-checker catches spelling mistakes on the fly.

![ide-type-error](https://cdn-images-1.medium.com/max/1200/1*0K13YjrBJYUlJHcNiJiV6g.png)

Credit: https://medium.com/@ageitgey/learn-how-to-use-static-type-checking-in-python-3-6-in-10-minutes-12c86d72677b

# Type Checked Lettuce

In this lecture, we will study types and endow Lettuce with a static type system. Static type checking allows us to ensure that a variety of runtime errors will never happen when we execute the program.

- When we evaluate an identifier, we would like it to be defined (or bound) in the environment.
- Whenever we evaluate arithmetic expressions involving Plus, Minus, Mult, Div, ... we would like  to guarantee that the arguments evaluate to numbers.
- Whenever we evaluate comparisons like Geq, Eq, .., both arguments evaluate to numbers.
- Whenever we evaluate Boolean operators such as And, Or, Not, we would like its arguments to be booleans.
- If then else expressions must have a Boolean type condition and the then/else parts must have the same type.
- Whenever we call a function, the called function expression must evaluate to a closure. The argument must evaluate to an appropriate input type for the function.

## Lettuce Language

Let us consider Lettuce with numerical (double precision), Boolean, Closures, and __error__.  We will
not consider references yet. But include the _recursive function calls_. The grammar is shown below for your reference: 



$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Div(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Eq(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) & \text{if (expr) then expr else expr} \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) & \text{function (identifier-formal-parameter) expr } \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \text{function call - expr(expr)} \\
 & | & LetRec(\mathbf{Identifier}, \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr})  & \text{argument 1 - function name, argument 2 - parameter}\\
 &&& \text{argument 3 - function definition expression, argument 4 - body expr} \\[5pt]
\end{array}$$

## Examples of Welltyped vs. Mistyped Expressions. 

To motivate our study, let us start with various examples of welltyped vs. mistyped expressions (programs).

### 1. Mistyped Program 
~~~
let x = 10 in 
   if (x) then True else False
~~~

This is mistyped because x is used as a Boolean in the `if-then-else` statement.

### 2. Welltyped Program 

~~~
let x = 10 in 
   if (x >= 9) then 1 else 2
~~~

The type of the program as a whole is __num__ since it returns a numerical value.

### 3. Mistyped Program

~~~
let f =  function (x) 
    if (x >= 10) 
    then 9
    else false
 in f(10)
~~~

The program is considered mistyped since the function `f` does not have a definite return type: it returns a number or a boolean depending on the value of `x`. 

Note that such functions were perfectly OK thus far. So the decision to call them mistyped can be considered controversial. 

### 4. Mistyped Program

~~~
let f = 10 in 
   f (20) // This is FunCall
~~~

Although f is used as a called function in `f (20)`, note that it is not a function.

### 5. Welltyped Program

~~~
let rec f = function (x) f (x - 1) in 
   f(36)
~~~

We will accept this program as valid typed even the recursive call never terminates. Why do we make this decision? It is somewhat pragmatic. It will turn out that proving termination/non-termination is very hard problem
for computers (this is called the halting problem). 

So if we said  that only terminating program can be well  typed: checking whether a program is well typed will become equivalent to solving the halting problem, which is not easy. Therefore, we will side step this issue for now. The situation is not as dire as one may think. It is certainly possible to design systems in which only terminating recursions can be defined. However, this is outside the scope of our studies in this class.


### 6. Mistyped Program

~~~
let x = 10 in 
  let y = true in 
      x - y
~~~

Subtracting a boolean from a number.

### 7. Mistyped program

~~~
let x = y + 10 in 
   x
~~~

`y` is not defined in the very first line of the program.



## Basic Types 

Let us study the problem of checking if a program is well typed by starting with the definition of all possible types. 

- __num__ is the type of all expressions that yield a numerical value in $\mathbb{R}$.
- __bool__ is the type of all expressions that yield a Boolean value in $\mathbb{B}$.
- __t1 => t2__ denotes a function type, wherein t1 and t2 are types.

We will skip a type for __error__. It is a value that will only be encountered at runtime and we will never
intentionally create __error__ in our language. So it will make no sense to have it as a valid type in our type system.

Let us take some examples:

- `Const(10.0)` has type __num__
- `Const(True)` has type __bool__
- `Plus(Const(10), Const(5))` has type __num__
- `Minus(Const(10), Const(True))` has no type in our system since it is a mistyped epression.
- `And( Geq(Const(10.0), Const(5.0)), Eq(Const(True), Const(False)) )` has type __bool__
- `Let("x", Const(15.0), Plus(Ident("x"), Const(20)))` has type __num__. 
- `FunDef("x", Plus(Ident("x"), Const(10)))` has type __num => num__  or is mistyped, depending on the type of the identifier "x".  

The last example already shows us the main issue. We cannot really talk about the type of an expression unless, we know the type of identifiers. 

In the first version of Lettuce, we will use type annotations.

# Lettuce with Type Annotations

Type annotations are common in many languages such as Java, C/C++, and Scala. For instance, consider the program below:

~~~
int foo(int y){
  int x = 200;
  return y + x 
  }
~~~

The type annotations tell us to expect that 
- y is an integer
- x is an integer
- The function foo takes one integer argument and returns an integer.

This does not mean that the function is well typed. However, having declared the types, the compiler can
more easily check the function and make sure that the declared types match with what the program does.

Similar type annotations are used in Scala programs.

~~~
def f(y: Int): Int = {
    val x : Int = 200
    y + x
}
~~~

We will do the same with Lettuce with type annotations. The syntax is different but the idea is the same.


## Where should type annotations be added?

Let us consider the basic problem of where type annotations should be added in a Lettuce program. 
One way of doing this is to make sure that every identifier that is introduced also has a type annotations.

There are two places where new identifiers are introduced:

- Let bindings: `let x = .... in ...`. We should put in a type annotation for x.
- Ditto for Let rec bindings.
- Function call parameters: ` function (x) ... `.  We should put in a type annotation for x.

Therefore, we will modify Lettuce syntax to allow type annotations at the site of let bindings, let rec bindings and function call definitions.

## Grammar of Types

We will first start with a grammar for the types that can be annotated.

$$\begin{array}{rcl}
\mathbf{Type} & \rightarrow & NumType \\
& | & BoolType \\
& | & FunType(\mathbf{Type}, \mathbf{Type}) \\
\end{array} $$

The grammar is simple enough: we have types for numbers `NumType`, booleans `BoolType`,  and
functions `FunType(t1, t2)`.

A complex type such as __ num => ( (num => bool) => bool) __ is expressed as `FunType(NumType, FunType( FunType(NumType, BoolType), BoolType))`.

## Lettuce Grammar with Type Annotations

We will first explain the concrete syntax with type annotations.

- Let bindings with type annotations will have the form 
   ~~~ 
   let x: type = expr in expr 
   ~~~

- Function definition expressions will have the form
   ~~~
   function (x: type) ... expr ... 
   ~~~
   
- Letrec bindings with type annotations will have the form
   ~~~
    let rec f: type = function (x: type) expr in expr
   ~~~
   

$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Eq(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) & \text{if (expr) then expr else expr} \\
 & | & Let( \mathbf{Identifier}, \color{red}{\mathbf{Type}}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
 & | & FunDef( \mathbf{Identifier}, \color{red}{\mathbf{Type}}, \mathbf{Expr}) & \text{function (identifier-formal-parameter) expr } \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \text{function call - expr(expr)} \\
 & | & LetRec(\mathbf{Identifier}, \color{red}{\mathbf{Type}}, \mathbf{Identifier}, \color{red}{\mathbf{Type}}, \mathbf{Expr}, \mathbf{Expr})  & \text{}\\
 &&& \text{} \\[5pt]
\end{array}$$


### Examples of type annotated programs

__Example 1:__  The concrete syntax is 
~~~
let x : num = 25 in 
    x + 30
~~~
The abstract syntax is 
~~~
Let("x", NumType, Const(25), Plus(Ident("x"), Const(30)) )
~~~

__Example 2:__ Concrete syntax
~~~
let y : num = 15 in 
 let x : bool = 25 >= y in 
     x and (30 >= y)
~~~

Abstract syntax
~~~
Let ("y", NumType, Const(15), 
        Let ("x", BoolType, Geq (Const(25), Ident("y")), 
              And( Ident("x"), Geq(Const(30), Ident("y")) )
             )
    )
~~~

__Example 3:__ Concrete Syntax

~~~
let f : (num => num) => num = function (g: num => num) g(20) in 
    let g : num => num = function (x: num) x in 
        f (g) 
~~~

Abstract Syntax
~~~
Let ("f", FunType( FunType(NumType, NumType), NumType ), 
          FunDef("g", FunType(NumType, NumType), FunCall( Ident("g"), Const(20) ), 
          Let ("g", FunType(NumType, NumType), FunDef("x", Num, Ident("x")), 
              FunCall(Ident("f"), Ident("g"))
~~~

We are ready to add the AST to scala in the usual manner.

In [3]:
sealed trait Type
case object NumType extends Type
case object BoolType extends Type
case class FunType(t1: Type, t2: Type) extends Type

sealed trait Program
sealed trait Expr

case class Const(f: Double) extends Expr
case class Ident(s: String) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr
case class Div(e1: Expr, e2: Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr
case class Geq(e1: Expr, e2: Expr) extends Expr
case class IfThenElse(e1: Expr, e2: Expr, e3: Expr) extends Expr
case class Let(x: String, xType: Type, e1: Expr, e2: Expr) extends Expr
case class FunDef(id: String, idType: Type, e: Expr) extends Expr
case class FunCall(calledFun: Expr, argExpr: Expr) extends Expr
case class LetRec(funName: String, funType: Type, param: String, paramType: Type, funExpr: Expr, bodyExpr: Expr) extends Expr

case class TopLevel(e: Expr) extends Program


defined trait Type
defined object NumType
defined object BoolType
defined class FunType
defined trait Program
defined trait Expr
defined class Const
defined class Ident
defined class Minus
defined class Plus
defined class Mult
defined class Div
defined class Eq
defined class Geq
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall
defined class LetRec
defined class TopLevel

## Type Checking

Given a type annotated lettuce program, is the type annotation correct? To do so, we are going to implement a type checker that uses and checks the type annotation. Thus, we should be able to say things like this:

### Example 1

The program 
~~~
let x: num = true in 
   x + 10
~~~
is mistyped.

The program 
~~~
let x: num = 25 in 
   x + 10
~~~
is welltyped and has type `num`.

### Example 2

The program 
~~~
let rec f: num => num = function (x: num) if (x <= 0) then 1 else x * f(x-1) in 
  f(30)
~~~
is welltyped and has type `num`.

The program 
~~~
let rec f: num => num = function (x: num) if (x <= 0) then true else x >= f(x-1) in 
  f(30)
~~~
is mistyped.

The goal of a type checker is to take in an annotated lettuce program and compute its type. If the program's type is successfully computed, we also conclude that it is well typed and in doing so, we calculate the type of each expression. Therefore, we will define a recursive type checker that calculates the type of an expression:
`typeOf(expr, type-environment)` much like an interpreter implements a function `eval(expr, environment)`.

- We will need to clarify what a `type-environment` is in contrast to the environments we are familar with.
- `typeOf` is either successful and provides a type for the expression or computes a __type-error__ that signifies a mistyped expression.

### Errors that a type checker will not catch.

As mentioned earlier, there are runtime errors that a type checker can catch before we run the program. On the other hand, there are errors that we cannot catch using a type checker. Let us clearly mention what these are:
- Nontermination -- we can have programs that may not terminate. We cannot catch these errors.
- Divide by zero, log on negative values, etc.. These kinds of errors are not type errors. When we write a program: 
~~~ 
let x : num = 0 in 
  let f : num => num = function (y: num) y/x in 
      f(220)
~~~

The program is typed correctly. However, the fact that it divides by zero is not part of our type system and we will note that it is in general undecidable. We will call these runtime errors.


### Types and Values


First we will denote for each type in our system, a set of values that belong to that type:
- __num__ corresponds to all values in $\mathbb{R}$.
- __bool__ corresponds to all values in $\mathbb{B}$.
- __t1 => t2__ corresponds to all closures in $\mathbb{C}$ that have a formal argument that takes in value `t1` and returns a value of type `t2`.

### Type Environments

Just like we have environments $\sigma$ that map identifiers to values, we need to have type environments $\alpha$ that map identifiers to types.

Example: 

- Enviromnent : $\sigma: \{ "x" \mapsto 2, "y" \mapsto true, "f" \mapsto Closure("x", Plus("x", Const(10))) \}$.
- Type Environment: $\alpha: \{ "x" \mapsto \mathbf{num},\ "y" \mapsto \mathbf{bool},\ "f" \mapsto \mathbf{num} => \mathbf{num} \}$.

We say that an environment $\sigma$ is _compatible_ with a type environment $\alpha$ if and only if 
- $\mathbf{domain}(\sigma) \subseteq \mathbf{domain}(\alpha)$ - everything defined in $\sigma$ must be typed in $\alpha$.
- $\sigma(x)$ is a value of type $\alpha(x)$ for every $x \in \mathbf{domain}(\sigma)$. If $v$ is the value of $x$ in $\sigma$, the type in $\alpha$ must be the type of value $v$.

**Definition** An expression `e` in a program has a type `t`  under type environment $\tau$ if and only if 
- for any environment $\sigma$ that is _compatible_ with $\alpha$, 
- evaluating the expression $e$ under $\sigma$ must yield one of the following possibilities:
   - A value of type $t$, 
   - Nonterminating execution, or
   - A runtime error due to division by zero.
   
   
We will use a special type value __typeerror__ to signify that an expression is mistyped. In practice, our type checker will simply throw an exception under type errors.

## Type Checking for Lettuce With Recursion

$$\newcommand\typeOf{\mathbf{typeOf}}$$
Let us first look at a type checker without recursion. We will write proof rules for the 
function $\typeOf(\texttt{e}, \alpha)$, the type of an expression under environment $\alpha$.

## Const Rule

$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array}\;(\text{#3}) }$$

$$\semRule{}{\typeOf(\texttt{Const(f)}, \alpha) = \mathbf{num}}{Const}$$

## Identifier Rule

$$\semRule{x \in \mathbf{domain}(\alpha)}{ \typeOf(\texttt{Ident(x)}, \alpha)  = \alpha(x)}{ident-ok}$$
$$\semRule{x \not\in \mathbf{domain}(\alpha)}{ \typeOf(\texttt{Ident(x)}, \alpha)  = \mathbf{typeerror}}{ident-nok}$$

The first rule is straightforward. The second rule says that an undefined variable leads to a __typeerror__

## Arithmetic Rule

$$\semRule{\typeOf(\texttt{e1}, \alpha) = \mathbf{num},\ \typeOf(\texttt{e2}, \alpha) = \mathbf{num}, T \in \{ Plus, Minus, Mult, Div \} }{\typeOf(\texttt{T(e1, e2)}, \alpha) = \mathbf{num}}{arith-ok} $$

Note that we do not check if `Div` leads to a division by zero in our type checking rule. This is a difference between the interpreter and type checker.

Short circuit semantics for type error: 

$$\semRule{\color{red}{\typeOf(\texttt{e1}, \alpha) \not= \mathbf{num}},\ T \in \{ Plus, Minus, Mult, Div \} }{\typeOf(\texttt{T(e1, e2)}, \alpha) = \mathbf{typeerror}}{arith-nok-1} $$

Note that if `e1` leads to a result that is not a number, we simply shortcut.

$$\semRule{\typeOf(\texttt{e1}, \alpha) = \mathbf{num},\ \color{red}{\typeOf(\texttt{e2}, \alpha) \not= \mathbf{num}},\ T \in \{ Plus, Minus, Mult, Div \} }{\typeOf(\texttt{T(e1, e2)}, \alpha) = \mathbf{typeerror}}{arith-nok-2} $$

## Comparison Operator Rule

$$\semRule{\typeOf(\texttt{e1}, \alpha) = \mathbf{num},\ \typeOf(\texttt{e2}, \alpha) = \mathbf{num}, T \in \{ Geq\} }{\typeOf(\texttt{T(e1, e2)}, \alpha) = \mathbf{bool}}{geq-ok} $$

$$\semRule{\typeOf(\texttt{e1}, \alpha) = \typeOf(\texttt{e2}, \alpha) }{\typeOf(\texttt{Eq(e1, e2)}, \alpha) = \mathbf{bool}}{eq-ok} $$

Notice how the equality operator is unlike the Geq operator. We are allowed to compare any two types using the equality operator in our language.

__Exercise__ Write down the type error rules for comparsion operators.

## Boolean Operator Rule

$$\semRule{\typeOf(\texttt{e1}, \alpha) = \mathbf{bool},\ \typeOf(\texttt{e2}, \alpha) = \mathbf{bool}, T \in \{ And, Or\} }{\typeOf(\texttt{T(e1, e2)}, \alpha) = \mathbf{bool}}{boolop-ok} $$

Note that unlike the rule for interpreter, the type checker does not __short circuit__ boolean operators. This is because, we cannot know in advance if the value of `e1` is true for an `Or` or false for an `And`.

__Exercise__ Write down the type error rules for boolean operators.


## If Then Else Rule


$$\semRule{\typeOf(\texttt{condExpr}, \alpha) = \mathbf{bool}\\ \typeOf(\texttt{thenExpr}, \alpha) = \typeOf(\texttt{elseExpr}, \alpha) = t,\ t \not= \mathbf{typeerror} }{\typeOf(\texttt{If(condExpr, thenExpr, elseExpr)}, \alpha) = t}{ite-ok} $$

$$\semRule{\color{red}{\typeOf(\texttt{condExpr}, \alpha) \not= \mathbf{bool}} }{\typeOf(\texttt{If(condExpr, thenExpr, elseExpr)}, \alpha) = \mathbf{typeerror} }{ite-nok-1} $$

$$\semRule{\typeOf(\texttt{condExpr}, \alpha) = \mathbf{bool}\\ \color{red}{\typeOf(\texttt{thenExpr}, \alpha) \not= \typeOf(\texttt{elseExpr}, \alpha)}}{\typeOf(\texttt{If(condExpr, thenExpr, elseExpr)}, \alpha) = \mathbf{typeerror}}{ite-nok-2} $$

## Let bindings

$$\semRule{\typeOf(\texttt{xExpr}, \alpha) = \texttt{xType} \\
\typeOf(\texttt{body}, \alpha[\texttt{x} \mapsto \texttt{xType}]) = t}{ \typeOf(\texttt{Let(x, xType, xExpr, body)}, \alpha) =  t}{let-ok}$$

The rule has couple of things to note:
- Note how we check that `xExpr` must have type `xType`, the same type as the annotation for `x`.
- When type checking `body`, we add the mapping $x \mapsto \texttt{xType}$ to the type environment.

A type error results when `xExpr` does not match the type annotation for `x` in the binding.

$$\semRule{\color{red}{\typeOf(\texttt{xExpr}, \alpha) \not= \texttt{xType}}}{ \typeOf(\texttt{Let(x, xType, xExpr, body)}, \alpha) = \mathbf{typeerror}}{let-nok-1}$$

Similarly, a type error results when `body` leads to a type error.

$$\semRule{\typeOf(\texttt{xExpr}, \alpha) = \texttt{xType}\\
\color{red}{\typeOf(\texttt{body}, \alpha[x \mapsto \texttt{xType}]) = \mathbf{typeerror}}}{ \typeOf(\texttt{Let(x, xType, xExpr, body)}, \alpha) = \mathbf{typeerror} }{let-nok-2}$$

Note that having written down this rule, we are going to get the same effect from the let-ok rule already.

## Function Definitions

$$\semRule{\typeOf(\texttt{body}, \alpha[\texttt{arg} \mapsto \texttt{argType}]) = \texttt{returnType},\ \texttt{returnType} \not= \mathbf{typeerror}}{
\typeOf(\texttt{FunDef(arg, argType, body)}) = (\texttt{argType} => \texttt{returnType})}{fun-ok}$$


$$\semRule{\color{red}{\typeOf(\texttt{body}, \alpha[\texttt{arg} \mapsto \texttt{argType}]) =  \mathbf{typeerror}}}{
\typeOf(\texttt{FunDef(arg, argType, body)} = \mathbf{typeerror} }{fun-nok}$$

## Function Calls

$$\semRule{\typeOf(\texttt{argExpr}, \alpha) = \texttt{t1} \\ \typeOf(\texttt{funcExpr}, \alpha) = \texttt{t1} => \texttt{t2}}{\typeOf(\texttt{FunCall(funcExpr, argExpr)}, \alpha) = \texttt{t2}}{funcall-ok}$$

Note that technically in our type system when we have a function type `t1 => t2`, we know for a fact
that `t1` and `t2` cannot be a type error. This makes it redundant to say `t1 != typeerror` or `t2 != typeerror` in the rule.

$$\semRule{\typeOf(\texttt{argExpr}, \alpha) = \texttt{t1}\\ \color{red}{\typeOf(\texttt{funcExpr}, \alpha) \not= \texttt{t1} => \texttt{t2}}}{\typeOf(\texttt{FunCall(funcExpr, argExpr)}, \alpha) = \mathbf{typeerror}}{funcall-nok}$$

__Exercise__ Write rules when evaluating `funcExpr` or `argExpr` leads to a typeerror.

## Handling Recursion 

The rule for recursion is actually much easier than the corresponding rule when we wrote an interpreter to support it.

$$\semRule{\typeOf(\texttt{funcBody}, \alpha[arg \mapsto \texttt{argType}, f \mapsto \texttt{argType => returnType}]) = \texttt{t2}\\
\typeOf(\texttt{letBody}, \alpha[f \mapsto \texttt{argType => returnType}]) = t}{
\typeOf(\texttt{LetRec(f, argType => returnType, arg, argType, funcBody, letBody)},\alpha) = t}{rec-ok}$$

The idea behind the rule is as follows. Take a let rec definition:

~~~
let f: argType => returnType = function (x: argType) funcBody
   in letBody
~~~

- We check that if `f` were typed as `argType => returnType` and `arg` were typed as `argType`, does `funcBody` have a type of `returnType`?
- If yes to the question above, then we evaluate the type of `letBody` with `f` now receiving the type `argType => returnType`.




In [4]:
def typeEquals(t1: Type, t2: Type): Boolean = t1 == t2


case class TypeErrorException(s: String) extends Exception


defined function typeEquals
defined class TypeErrorException

In [13]:
def typeOf(e: Expr, alpha: Map[String, Type]): Type = {
    def checkType(opName: String, e1: Expr, t1: Type, e2: Expr, t2: Type, resType: Type): Type = {
        val t1hat = typeOf(e1, alpha)
        if (! typeEquals(t1hat, t1)){
            throw new TypeErrorException(s"Type mismatch in arithmetic/comparison/bool op $opName, Expected type $t1, obtained $t1hat")
        }
        
        val t2hat = typeOf(e2, alpha)
        if (! typeEquals(t2hat, t2)){
            throw new TypeErrorException(s"Type mismatch in arithmetic/comparison/bool op $opName, Expected type $t2, obtained $t2hat")
        }
        
        resType
    }
    
    e match {
        case Const(f) => NumType
        case Ident(s) => {if (alpha contains s)
                             alpha(s)
                          else 
                             throw TypeErrorException(s"Unknown identifier $s")}
        case Plus(e1, e2) =>  checkType("Plus", e1,  NumType, e2, NumType, NumType)
        case Minus(e1, e2) => checkType("Minus",e1,  NumType, e2, NumType, NumType)
        case Mult(e1, e2) => checkType("Mult",e1,  NumType, e2, NumType, NumType)
        case Div(e1, e2) => checkType("Div", e1,  NumType, e2, NumType, NumType)
        case Geq(e1, e2) => checkType("Geq", e1,  NumType, e2, NumType, BoolType)
        case Eq(e1, e2) => {
            val t1 = typeOf(e1, alpha)
            val t2 = typeOf(e2, alpha)
            if (typeEquals(t1, t2))
                BoolType
            else 
                throw TypeErrorException(s"Equality operator compares unequal types $t1 with $t2")
        }

        //case And(e1, e2) => checkType(e1, BoolType, e2, BoolType, BoolType)
        //case Or(e1, e2) => checkType(e1, BoolType, e2, BoolType, BoolType)
        case IfThenElse(e, e1, e2) => {
            val t = typeOf(e, alpha)
            if (t == BoolType){
                val t1 = typeOf(e1, alpha)
                val t2 = typeOf(e2, alpha)
                if (typeEquals(t1, t2))
                    t1
                else 
                    throw TypeErrorException(s"If then else returns unequal types $t1 and $t2")
            } else {
                throw TypeErrorException(s"If then else condition expression not boolean $t")
            }
        }

        case Let(x, t, e1, e2) => {
            val t1 = typeOf(e1, alpha)
            if (typeEquals(t1, t)){
                val newAlpha = alpha + (x -> t)
                typeOf(e2, newAlpha)
            } else {
                throw TypeErrorException(s"Let binding has type $t whereas it is bound to expression of type $t1")
            }
        }

        case FunDef(x, t1, e) => {
            val newAlpha = alpha + (x -> t1)
            val t2 = typeOf(e, newAlpha)
            FunType(t1, t2)
        }

        case FunCall(e1, e2) => {
            val ftype = typeOf(e1, alpha)
            ftype match {
                case FunType(t1, t2) => {
                    val argType = typeOf(e2, alpha)
                    if (typeEquals(argType, t1)){
                        t2
                    } else {
                        throw TypeErrorException(s"Call to function with incompatible argument type. Expected $t1, obtained $argType")
                    }
                }
                case _ => { throw TypeErrorException(s"Call to function but with a non function type $ftype")}

            }
        }

        case LetRec(f, fType, x, argType, e1, e2 ) => {
            fType match {
                case FunType(t1, t2) => {
                    if (typeEquals(argType, t1)){
                        val newAlpha = alpha +(f -> fType, x -> argType)
                        val retType = typeOf(e1, newAlpha)
                        if (typeEquals(retType, t2)){
                            typeOf(e2, alpha + (f -> fType))
                        } else {
                            throw TypeErrorException(s"Rec. function call expression evaluates to type $retType. Expected type: $t2")
                        }
                    } else {
                        throw TypeErrorException(s"Recursive function: function type $fType incompatible with declared type of argument: $argType")
                    }
                }
                case _ => {throw TypeErrorException(s"Recursive function is annotated with non function type: $fType")}
            }
        }
    }
}

def typeOfProgram(p: Program) = p match {
    case TopLevel(e) => {
        try {
            val t = typeOf(e, Map())
            println(s"Program type computed successfully as $t")
        } catch {
            case TypeErrorException(s) => println(s"Type error found: $s")
        }
    }
}

defined function typeOf
defined function typeOfProgram

In [14]:
val p1 = TopLevel(Let("x", NumType, Const(20), Plus(Ident("x"), Const(30)) ))
typeOfProgram(p1)

Program type computed successfully as NumType


p1: TopLevel = TopLevel(Let(x,NumType,Const(20.0),Plus(Ident(x),Const(30.0))))

In [16]:
/* let y = 15 in 
      let x = 25 >= y in 
          x >= (30 >= y)
          */
val p2 = TopLevel(Let ("y", NumType, Const(15), 
        Let ("x", BoolType, Geq (Const(25), Ident("y")), 
              Geq( Ident("x"), Geq(Const(30), Ident("y")) )
             )
    ))
typeOfProgram(p2)

Type error found: Type mismatch in arithmetic/comparison/bool op Geq, Expected type NumType, obtained BoolType


p2: TopLevel = TopLevel(Let(y,NumType,Const(15.0),Let(x,BoolType,Geq(Const(25.0),Ident(y)),Geq(Ident(x),Geq(Const(30.0),Ident(y))))))

In [18]:
val p = TopLevel(Let ("f", FunType( FunType(NumType, NumType), NumType ), 
          FunDef("g", FunType(NumType, NumType), FunCall( Ident("g"), Const(20) )), 
          Let ("g", FunType(NumType, NumType), FunDef("x", NumType, Ident("x")), 
              FunCall(Ident("f"), Ident("g")))))
typeOfProgram(p)

Program type computed successfully as NumType


p: TopLevel = TopLevel(Let(f,FunType(FunType(NumType,NumType),NumType),FunDef(g,FunType(NumType,NumType),FunCall(Ident(g),Const(20.0))),Let(g,FunType(NumType,NumType),FunDef(x,NumType,Ident(x)),FunCall(Ident(f),Ident(g)))))

In [21]:
/*
let rec f: Num => Num = function (z: Num) if (z <= 0) then 1 else 1 + f(z -1) in f(10)
*/

val p = TopLevel(
    LetRec("f", FunType(NumType, NumType), "z", NumType, 
          IfThenElse( 
                         Geq(Const(0), Ident("z")),
                         Const(1),
                         Plus(Const(1), FunCall(Ident("f"), Minus(Ident("z"), Const(1))))
                         ), 
                  FunCall(Ident("f"), Const(10))
          )

)
typeOfProgram(p)

Program type computed successfully as NumType


p: TopLevel = TopLevel(LetRec(f,FunType(NumType,NumType),z,NumType,IfThenElse(Geq(Const(0.0),Ident(z)),Const(1.0),Plus(Const(1.0),FunCall(Ident(f),Minus(Ident(z),Const(1.0))))),FunCall(Ident(f),Const(10.0))))